In [ ]:
# !pip install --upgrade plotly
# # !pip install jupyter-dash
# !pip install dash-bootstrap-components

In [ ]:
import dash
from dash import jupyter_dash
jupyter_dash.inline_exceptions = False
from dash import Dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
from dash import State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from PIL import Image
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import os
import re
import glob
import random
import plotnine
from sklearn import preprocessing
from tqdm import tqdm
import plotly.express as px
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
import pdb
# mount data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
dataPath = '/content/drive/MyDrive/Data/granularity/imageSequence/data'
IMAGE_FOLDER = '/content/drive/MyDrive/Data/granularity/imageSequence/images'
df = pd.read_csv(os.path.join(dataPath,'imageseq_data.csv'))
float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].apply(lambda x: np.round(x, 3))
df = df.sample(frac=1).reset_index(drop=True)
#df['name'] = df['name'].apply(lambda x: f"{x}.png")
first_image_name = df.iloc[0]['name']
first_image_path = f"{IMAGE_FOLDER}/{first_image_name}"
savedContainer = {'name':[], 'ratio':[], 'intensity':[], 'sd':[], 'maskArea':[], 'label':[]}
image_names = [re.sub("_.*","",name) for name in set(df.name.tolist())]
#df.loc[df.name.str.contains(image_names[0]),:]

In [ ]:
def openImageToPX(imageName):
  im_pil = Image.open(imageName)
  img = px.imshow(im_pil,binary_string=True, binary_backend="png", width=500, height=500,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
  return img

In [ ]:
import dash
from dash import html
from dash import dash_table, dcc
from dash.dependencies import Input, Output
#import dash_core_components as dcc
from dash import State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import os
import re
import glob
import random
from sklearn import preprocessing
from tqdm import tqdm
import plotly.express as px

dataPath = r'D:\Gil\images\pex_project\02142024\single_images\imageSequence\data'
IMAGE_FOLDER = r'D:\Gil\images\pex_project\02142024\single_images\imageSequence\images'
df = pd.read_csv(os.path.join(dataPath,'imageseq_data.csv'))
float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].apply(lambda x: np.round(x, 3))
df = df.sample(frac=1).reset_index(drop=True)
#df['name'] = df['name'].apply(lambda x: f"{x}.png")
first_image_name = df.iloc[0]['name']
first_image_path = f"{IMAGE_FOLDER}/{first_image_name}"
savedContainer = {'name':[], 'ratio':[], 'intensity':[], 'sd':[], 'maskArea':[], 'label':[]}
image_names = [re.sub("_.*","",name) for name in set(df.name.tolist())]

def openImageToPX(imageName):
  im_pil = Image.open(imageName)
  img = px.imshow(im_pil,binary_string=True, binary_backend="png", width=650, height=650,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
  return img

# im_pil = Image.open(os.path.join(IMAGE_FOLDER,df.loc[0,'name']))
# fig = px.imshow(im_pil,zmin=[50, 50, 50], zmax=[100, 100, 255],binary_string=True, binary_backend="png", width=650, height=650,binary_compression_level=9).update_xaxes(showticklabels=False).update_yaxes(showticklabels = False)
# #zmin=[50, 50, 50], zmax=[255, 255, 255]
# fig.show()


app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.layout = html.Div([
       html.Div([
        dash_table.DataTable(id='table',
        columns=[{"name": i, "id": i,}
                for i in df.columns if i in ['name', 'ratio', 'label']],
        data=df.to_dict('records'),
        row_selectable="single",
        editable = True,
        page_size=10,
        page_current = 0,
        page_action='native',
        row_deletable=True,
        cell_selectable=True)],style={'width': '49%', 'display': 'inline-block'}),
        html.Div([
          html.Div(id='img-container')
        ], style={'width': '49%', 'display': 'inline-block'}),
])

@app.callback(
  Output('img-container', 'children'),
  [Input('table', 'active_cell'),
  State('table', 'page_current'),
  State('table', 'page_size')],
  prevent_initial_call=True,)
def display_image(active_cell,page_current,page_size):
  '''
  active_cell - dictionery {'row': 0, 'column': 0, 'column_id': 'name'}
  rows - list of dict per row:  [{'Unnamed: 0': 294, 'name': 'QOnvDbK_24.png', 'ratio': 0.032,
  'intensity': 783.206, 'sd': 611.006, 'maskArea': 0.995, 'label': 0},....]
  '''
  if (active_cell is None) and (page_current == 0):
    img_name = df.loc[0,'name']
    img = openImageToPX(os.path.join(IMAGE_FOLDER,img_name))
    return [dcc.Graph(id="disp3",figure=img)]
  elif (active_cell is None) and (page_current > 0):
    return ["select image name"]
  elif (active_cell['column_id'] == 'name') and (page_current == 0):
    img_name = df.loc[active_cell['row'],active_cell['column_id']]
    img = openImageToPX(os.path.join(IMAGE_FOLDER,img_name))
    return [dcc.Graph(id="disp2",figure=img)]
  elif  (active_cell['column_id'] == 'name') and (page_current > 0):
    df_new = df.iloc[page_current*page_size:(page_current+1)*page_size].reset_index()
    img_name = df_new.loc[active_cell['row'],active_cell['column_id']]  # No need to append ".png"
    img = openImageToPX(os.path.join(IMAGE_FOLDER,img_name))
    return [dcc.Graph(id="disp1",figure=img)]

@app.callback(
  Output('temp-contain', 'children',allow_duplicate=True),
  [Input('table', 'active_cell'),
   State('table', 'data'),
   State('table', 'page_current'),
   State('table', 'page_size')],
  prevent_initial_call=True,
)
def update_data(active_cell,curr_table,page_current,page_size):
  # Handler for data update; includes save logic as needed
  if (active_cell is None) and (page_current == 0):
    return ""
  elif (active_cell is None) and (page_current > 0):
    return ""
  elif (active_cell['column_id'] == 'label') and (page_current==0):
    label = df.loc[active_cell['row'],'label']
    df.iloc[active_cell['row'],'label'] = label
    df.to_csv(os.path(dataPath,'imageseq_data.csv'))
    return ""
  elif (active_cell['column_id'] == 'label') and (page_current > 0):
    label = df.loc[active_cell['row'],'label']
    df.iloc[page_current*page_size+active_cell['row'],'label'] = label
    df.to_csv(os.path(dataPath,'imageseq_data.csv'))
    return ""
  
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)


